In [5]:
import sys
sys.path.append("/home/jovyan/work/Scripts")
from pyspark.sql.functions import col

In [6]:
from spark_session import create_spark_session
spark = create_spark_session()


In [7]:
ano = "2025"
bronze_path = f"s3a://datalake/bronze/sih_sus/{ano}/*.parquet"               
silver_path = f"s3a://datalake/silver/sih_sus/"

In [8]:
df_raw = spark.read.parquet(bronze_path)


In [9]:
# Selecionar colunas específicas
df_sel = df_raw.select("UF_ZI", "ANO_CMPT","MES_CMPT","MUNIC_RES", "SEXO", "IDADE", "DIAG_PRINC", "VAL_TOT", "MORTE","DT_INTER","DT_SAIDA")



In [14]:
df_sel = (
    df_sel
    .withColumnRenamed("UF_ZI", "uf")
    .withColumnRenamed("ANO_CMPT", "ano_competencia")
    .withColumnRenamed("MES_CMPT", "mes_competencia")
    .withColumnRenamed("MUNIC_RES", "municipio_id")
    .withColumnRenamed("SEXO", "sexo")
    .withColumnRenamed("IDADE", "idade")   
    .withColumnRenamed("DIAG_PRINC", "cid_principal")    
    .withColumnRenamed("VAL_TOT", "valor_total")
    .withColumnRenamed("MORTE", "morte")
    .withColumnRenamed("DT_INTER", "data_internacao")
    .withColumnRenamed("DT_SAIDA", "data_saida")    
)

In [15]:
try:
    df_sel.write.format("delta") \
        .mode("overwrite") \
        .partitionBy("ano_competencia", "mes_competencia") \
        .save(silver_path)
except Exception as e:
    print("Erro:", e)

In [16]:
spark.stop()